In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

label2id = {'negative':0,'positive':1}

df = pd.read_csv('/root/autodl-tmp/IPCK/dataset/dataset.csv')
df = df.replace({"label": label2id})
df.dropna(inplace=True)

/tmp/ipykernel_3023/712239689.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


In [19]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.1, random_state=42)

In [20]:
train

,Unnamed: 0,label,message,diff
1420,1420,0,Changes to ensure record data can wrap only on...,diff --git a/libevt/libevt_io_handle.c b/libev...
456,456,1,Fix carbons,diff --git a/slixmpp/plugins/xep_0280/carbons....
8981,8981,0,new setting page layout,diff --git a/includes/view/User_view.php b/inc...
6970,6970,1,Disable logging during brokerfactory de-serial...,diff --git a/openjpa-kernel/src/main/java/org/...
6959,6959,1,Fix acl_file being ignore for default listener...,diff --git a/ChangeLog.txt b/ChangeLog.txt\nin...
...,...,...,...,...
1382,1382,0,Merge pull request #3332 from square/jakew/ded...,diff --git a/retrofit-adapters/guava/src/main/...
4581,4581,1,mercurial: separate command options and positi...,diff --git a/lib/redmine/scm/adapters/mercuria...
2738,2738,0,"Land #14680, winrm_script_exec: print command ...",diff --git a/modules/exploits/windows/winrm/wi...
5074,5074,0,Merge pull request #363 from mateusz/uploadfie...,diff --git a/forms/UploadField.php b/forms/Upl...


In [21]:
# 假设你有一个DataFrame df，其中包含'commit_message'和'label'列
# df = pd.DataFrame({'commit_message': [...], 'label': [...]})

# 1. 数据预处理和特征提取
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['diff'])
y_train = train['label']

# 3. 训练XGBoost模型
clf = XGBClassifier()
clf.fit(X_train, y_train)

# 4. 评估模型
y_pred = clf.predict(vectorizer.transform(test['diff']))
print("Accuracy:", accuracy_score(test['label'], y_pred))
print("Classification Report:\n", classification_report(test['label'], y_pred,digits=4))


Accuracy: 0.8306010928961749
Classification Report:
               precision    recall  f1-score   support

           0     0.8430    0.9016    0.8713      1280
           1     0.8043    0.7067    0.7524       733

    accuracy                         0.8306      2013
   macro avg     0.8236    0.8041    0.8118      2013
weighted avg     0.8289    0.8306    0.8280      2013

